# 5.2 K-Nearest Neighbors for Regression

Linear regression is not the only machine learning model that Ashenfelter could have fit.

In [ ]:
import pandas as pd
import numpy as np

data_dir = "https://dlsun.github.io/pods/data/"
bordeaux_df = pd.read_csv(data_dir + "bordeaux.csv",
                          index_col="year")

# Split the data into training and test sets.
bordeaux_train = bordeaux_df.loc[:1980].copy()
bordeaux_test = bordeaux_df.loc[1980:].copy()

# Log transform the target.
bordeaux_train["log(price)"] = np.log(bordeaux_train["price"])
bordeaux_train.head()

Let's focus on just two features for now: winter rainfall (**win**) and average summer temperature (**summer**). Let's plot the training data, using a color gradient to represent the target (**log(price)**). Notice how we can customize the color gradient using the `cmap=` argument. A list of the available colormaps can be found [here](https://matplotlib.org/examples/color/colormaps_reference.html).

In [ ]:
import matplotlib.cm as cm

bordeaux_train.plot.scatter(x="win", y="summer", c="log(price)", 
                            cmap=cm.YlOrRd)

Notice how wines that are close on this scatterplot are of similar quality. This insight is the basis of the $k$-nearest neighbors algorithm for predicting wine quality. Suppose that we want to predict the quality of the 1986 vintage, represented by a blue star in the plot below. 

![](regression_neighbors.png)

The $k=5$ points that are closest to this point in feature space are indicated by dotted lines. We can average the qualities of thse wines to obtain our prediction. All 5 of these points have a quality less than 3.0, so the 1986 vintage is also predicted to be of low quality.

The $k$-nearest neighbors regression algorithm can be summarized as follows:

1. Determine the $k$ closest points in the training data to the new point that you want to predict for, based on some distance metric on the features.
2. The predicted label of the new point is the mean (or median) of the labels of the $k$ closest points.

Let's implement this in code. First, we extract the training data and scale the features:

In [ ]:
X_train = bordeaux_train[["win", "summer"]]
y_train = bordeaux_train["log(price)"]

# Standardize the features.
X_train_mean = X_train.mean()
X_train_sd = X_train.std()
X_train_st = (X_train - X_train_mean) / X_train_sd

Now, we get the features for the new observation (i.e., the 1986 vintage), standardizing it in the same way.

In [ ]:
x_new = bordeaux_test.loc[1986, ["win", "summer"]]

x_new_st = (x_new - X_train_mean) / X_train_sd
x_new_st

Now we calculate the (Euclidean) distances between the 1986 vintage and the vintages in the training data.

In [ ]:
dists = np.sqrt(((X_train_st - x_new_st) ** 2).sum(axis=1))
dists

Now, we sort the distances. The first 5 of these are the nearest neighbors. To get the year of these nearest neighbors, we get the index.

In [ ]:
i_nearest = dists.sort_values().index[:5]
i_nearest

We can look up these years in the training data.

In [ ]:
bordeaux_train.loc[i_nearest]

To make a prediction for the price of the 1986 vintage, we average the sale prices of these 5-nearest neighbors.

In [ ]:
y_train.loc[i_nearest].mean()

So the model predicts that the quality of the 1986 is about 2.56, which is well below the average quality.

Of course, the model above only had two features so it was easy to visualize the "nearest neighbors" on the scatterplot. The magic of $k$-nearest neighbors is that it still works when there are more features and the data is not possible to visualize.

## K-Nearest Neighbors in scikit-learn

Now let's see how to implement $k$-nearest neighbors in scikit-learn. Recall from the previous lesson that all scikit-learn models follow the three-step pattern:

1. Declare the model.
2. Fit the model to training data.
3. Use the model to predict on test data.

To fit a $k$-nearest neighbors model instead of a linear regression model, we only need to modify the first step. Instead of declaring a model of type `LinearRegression`, we define a model of type `KNeighborsRegressor`, specifying the value of $k$ as one of the parameters. Because `KNeighborsRegressor` works with distances, it is a good idea to scale the features before passing the features into the model. (Refer back to Chapter 3 for a full explanation.)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

# Standardize the training and test data
scaler = StandardScaler()
X_train_st = scaler.fit_transform(bordeaux_train[["win", "summer"]])
X_new_st = scaler.transform(bordeaux_test.loc[[1986], ["win", "summer"]])
y_train = bordeaux_train["log(price)"]

# Fit k-nearest neighbors
model = KNeighborsRegressor(n_neighbors=5)
model.fit(X=X_train_st, y=y_train)
model.predict(X=X_new_st)

This is the same predicted value that we got by implementing $k$-nearest neighbors manually.

## The K-Nearest Neighbors Regression Function

A predictive model is simply a function $f$ that maps feature values ${\bf x}$ to target values $y$. We can visualize $f$ when ${\bf x}$ consists of just a single feature, such as **age**. In the previous lesson, we saw that $f$ is just a line when the model is linear regression. What does $f$ look like when the model is a $k$-nearest neighbors regressor?

First, we extract the training data. There is no need to scale the features in this case because there is only one feature. (The point of scaling is to bring all of the variables to the same scale.

In [ ]:
X_train = bordeaux_train[["age"]]
y_train = bordeaux_train["log(price)"]

Now we fit the $k$-nearest neighbor model as before. We do not need to standardize the feature in this case because there is only one feature. Standardizing is only useful when there are multiple features that we want to bring to the same scale.

In [ ]:
# Fit k-nearest neighbors
model = KNeighborsRegressor(n_neighbors=5)
model.fit(X=X_train, y=y_train)

In order to graph $f$, we need to evaluate the predictive model at a grid of feature values. Since age ranges from 12 to 40 in the training data, we create a grid of ${\bf x}$ values from 10 to 45, make predictions at these values, and plot these predictions as a curve.

In [ ]:
# Define a grid of feature values.
X_new = pd.DataFrame()
X_new["age"] = np.linspace(10, 45, num=200)

# Make predictions at those feature values.
y_new_ = pd.Series(
    model.predict(X_new),
    index=X_new["age"]
)

# Plot the predictions.
bordeaux_train.plot.scatter(x="age", y="log(price)")
y_new_.plot.line()

Unlike the linear regression model, the $k$-nearest neighbor model is piecewise constant. For example, wines more than 37 years old all have the same 5-nearest neighbors, so the prediction is constant in that range.

# Exercises

1\. Plot the $k$-nearest neighbors regression function for predicting the quality of a wine from its age for $k=1, 5, 10, 20$. How does the regression function change as $k$ increases?

2\. Using the Ames housing data set (http://dlsun.github.io/pods/data/AmesHousing.txt ), fit a $k$-nearest neighbors model that predicts the price of a home using square footage, number of bedrooms (**Bedroom AbvGr**), number of full bathrooms (**Full Bath**), and number of half bathrooms (**Half Bath**). Then, use your fitted model to predict the price of a home that is 1500 square feet, with 3 bedrooms, 2 full baths, and 1 half bath.

3\. You would like to predict how much a male diner will tip on a bill of \\$40.00 on a Sunday. Build a $k$-nearest neighbors model to answer this question, using the tips data (http://dlsun.github.io/pods/data/tips.csv ) as your training data.